# Experimenting with the RLSD Algorithm

In [ ]:
function RLSD(
    model,
    train::Function,
    transition::Function,
    reward,
    observation,
    T, # temperature
    epochs::Int64,
    n::Int64)
    beta = 0
    for epoch in epochs
        R = []
        Dsynth = []
        Dtrain = []
        for i in 1:n
            o = observation(s)
            p = model(o)
            (rand() < q) ? push!(Dsynth, (o, p))
            a = argmax(p)
            (rand() < q) ? push!(R, reward(s_next, a))
            s_next = transition(s, a)
            s = s_next
        end
        R = mean(R)

        if (R > beta) # better than baseline: make highest action candidate likelier
            Dtrain = []
            for (o, p) in Dsynth
                y = softmax(p, T*(R - beta)/beta)
                push!(Dtrain, (o, y))
            end
            train(model, Dtrain)
        else
            Dtrain = []
            for (o, p) in Dsynth
                y = explore(p, (beta-R)/beta)
                push!(Dtrain, (o, y))
            end
            train(model, Dtrain)
        end
    end
    beta = ((epoch-1)beta + R)/epoch 
end


In [ ]:
import Flux
import Random
import Statistics

In [ ]:
function explore(v, Δ)
    return v + Δ(v - ones(length(v)))
end

In [ ]:
function train(model, data)
    train!((x,y) -> crossentropy(model(x), y), Flux.params(model), data, Adam())
end

In [ ]:
model = Flux.Chain(
    Dense(inp_size, hl_size, relu),
    [Dense(hl_size, hl_size, relu) for i in 1:nb_hl]...,
    Dense(hl_size, out_size, s),
    softmax)